In [2]:
import sys
from pathlib import Path

In [3]:
!{sys.executable} -m pip install flair

  Using cached lxml-5.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached regex-2023.12.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.10.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 6.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━

In [26]:
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.models import TARSClassifier
from flair.trainers import ModelTrainer

In [29]:
tars = TARSClassifier.load("tars-base")

2024-02-17 00:47:23,715 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model


In [32]:
from flair.data import Corpus
from flair.datasets import FlairDatapointDataset
# training dataset consisting of four sentences (2 labeled as "olahraga" (sports) and 2 labeled as "politik" (politics)
train = FlairDatapointDataset(
    [
        Sentence('BSN Sports LLC')
        .add_label('payee_or_misc', 'payee'),
        Sentence('Samuel L. Catherin')
        .add_label('payee_or_misc', 'payee'),
        Sentence('Five Seasons')
        .add_label('payee_or_misc', 'payee'),
        Sentence('fifty thousand and 00/11')
        .add_label('payee_or_misc', 'misc'),
        Sentence('Official check')
        .add_label('payee_or_misc', 'misc')
    ])
test = FlairDatapointDataset(
    [
        Sentence('Central hudson electric').add_label('payee_or_misc', 'payee'),
        Sentence('Pay to the order of').add_label('payee_or_misc', 'misc')
    ])
# make a corpus with train and test split
corpus = Corpus(train=train, test=test)

2024-02-17 00:52:29,947 No dev split found. Using 0% (i.e. 0 samples) of the train split as dev data


In [36]:
corpus.make_label_dictionary(label_type="payee_or_misc")

2024-02-17 00:54:53,433 Computing label dictionary. Progress:


0it [00:00, ?it/s]
5it [00:00, 12710.01it/s]

2024-02-17 00:54:53,439 Dictionary created for label 'payee_or_misc' with 2 values: payee (seen 3 times), misc (seen 2 times)


In [39]:
from flair.trainers import ModelTrainer
# 1. load base TARS
tars = TARSClassifier.load('tars-base')
# 2. make the model aware of the desired set of labels from the new corpus
tars.add_and_switch_to_new_task("PAYEE_OR_MISC", 
                                label_dictionary=corpus.make_label_dictionary("payee_or_misc"),
                                label_type="payee_or_misc")
# 3. initialize the text classifier trainer with your corpus
trainer = ModelTrainer(tars, corpus)

2024-02-17 00:55:25,508 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model
2024-02-17 00:55:25,567 Computing label dictionary. Progress:


0it [00:00, ?it/s]
5it [00:00, 12045.67it/s]

2024-02-17 00:55:25,573 Dictionary created for label 'payee_or_misc' with 2 values: payee (seen 3 times), misc (seen 2 times)


In [40]:
# 4. train model
trainer.train(base_path='resources/taggers/payee', # path to store the model artifacts
              learning_rate=0.02, 
              mini_batch_size=1, 
              max_epochs=10, 
              )

2024-02-17 00:55:51,709 ----------------------------------------------------------------------------------------------------
2024-02-17 00:55:51,711 Model: "TARSClassifier(
  (tars_model): TextClassifier(
    (embeddings): TransformerDocumentEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Lin

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 89.99it/s]

2024-02-17 00:55:57,553 
Results:
- F-score (micro) 0.5
- F-score (macro) 0.3333
- Accuracy 0.5

By class:
              precision    recall  f1-score   support

       payee     0.5000    1.0000    0.6667         1
        misc     0.0000    0.0000    0.0000         1

    accuracy                         0.5000         2
   macro avg     0.2500    0.5000    0.3333         2
weighted avg     0.2500    0.5000    0.3333         2

2024-02-17 00:55:57,554 ----------------------------------------------------------------------------------------------------


{'test_score': 0.5}

In [41]:
# 5. Load few-shot TARS model
tars = TARSClassifier.load('resources/taggers/payee/final-model.pt')

In [56]:
# 6. Prepare a test sentence
sentence = Sentence("Weber County Treasurer")
# 7. Predict for olahraga and politik
tars.predict(sentence, return_probabilities_for_all_classes=True)
print(sentence)

Sentence[3]: "Weber County Treasurer"
